<b><h3>Mount Drive</h3></b>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<b><h3>Import Libraries</h3></b>

In [ ]:
import numpy as np
import pandas as pd

<b><h3>Import Dataset</h3></b>

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/PowerLoadDataset.csv')
data.head()

,Date,Time,Load (kW)
0,01.09.2018,0:00,5551.82208
1,NaN,1:00,4983.17184
2,NaN,2:00,4888.39680
3,NaN,3:00,5072.95872
4,NaN,4:00,5196.25980


<b><h3>ReModelling Dataset</h3></b>

In [ ]:
x = []
y = []
for i in range(24, data.shape[0]):
  x.append(data['Load (kW)'].iloc[i-24])
  y.append(data['Load (kW)'].iloc[i])
data = pd.DataFrame({'Previous Day Load' : x, 'Present Day Load' : y})
data.head()

,Previous Day Load,Present Day Load
0,5551.82208,4931.26380
1,4983.17184,4775.53968
2,4888.39680,4713.81120
3,5072.95872,4689.02628
4,5196.25980,4844.75040


<b><h3>Data Normalisation</h3></b>

In [ ]:
Xmax = np.max(data['Previous Day Load'])
Xmin = np.min(data['Previous Day Load'])
Ymax = np.max(data['Present Day Load'])
Ymin = np.min(data['Present Day Load'])
data['Previous Day Load'] = (data['Previous Day Load'] - Xmin) / (Xmax - Xmin) 
data['Present Day Load'] = (data['Present Day Load'] - Ymin) / (Ymax - Ymin)
data.head()

,Previous Day Load,Present Day Load
0,0.397877,0.284300
1,0.293800,0.255799
2,0.276454,0.244501
3,0.310234,0.239965
4,0.332801,0.268466


<b><h3>Null Values Treatment</h3></b>

In [ ]:
print(data.isnull().sum())
m1=data['Previous Day Load'].mean()
m2=data['Present Day Load'].mean()
data['Previous Day Load'].fillna(value=m1, inplace=True)
data['Present Day Load'].fillna(value=m2, inplace=True)

Previous Day Load    0
Present Day Load     0
dtype: int64


<b><h3>Split The Data</h3></b>

In [ ]:
from sklearn.model_selection import train_test_split
x = data['Previous Day Load']
y = data['Present Day Load']
Xtrain, Xtest, Ytrain, Ytest = train_test_split(x, y, test_size=0.1, random_state=100)

<b><h3>Gradient Calculation</h3></b>

In [ ]:
m = np.random.uniform(-5, 5)
c = np.random.uniform(-5, 5)
eta = 0.1
epochs = 500
vm = 0
vc = 0
v = 0.1

for _ in range(epochs):
  for i in range(Xtrain.shape[0]):
     tempM = m + v*vm
     tempC = c + v*vc
     gradM = -1 * (Ytrain.iloc[i] - tempM * Xtrain.iloc[i] - tempC) * Xtrain.iloc[i]
     gradC = -1 * (Ytrain.iloc[i] - tempM * Xtrain.iloc[i] - tempC)
     vm = (v * vm) - (eta * gradM)
     vc = (v * vc) - (eta * gradC)
     m += vm
     c += vc

print('Values of Model Parameters m and c : ',m, c)

Values of Model Parameters m and c :  0.7547882346894 0.10550820212384142


<b><h3>Data Denormalisation</h3></b>

In [ ]:
train_pred = [m * Xtrain.iloc[i] + c for i in range(Xtrain.shape[0])]
test_pred = [m * Xtest.iloc[i] + c for i in range(Xtest.shape[0])]

train_pred = [i * (Ymax - Ymin) + Ymin for i in train_pred]
test_pred = [i * (Ymax - Ymin) + Ymin for i in test_pred]
Ytrain = [i * (Ymax - Ymin) + Ymin for i in Ytrain]
Ytest = [i * (Ymax - Ymin) + Ymin for i in Ytest]

<b><h3>Error Metrics Calculation</h3></b>

In [ ]:
training_MAE = sum([abs(train_pred[i] - Ytrain[i]) for i in range(len(Ytrain))]) / len(Ytrain)
training_MSE = sum([(train_pred[i] - Ytrain[i]) ** 2 for i in range(len(Ytrain))]) / len(Ytrain)
training_RMSE = training_MSE ** 0.5
print('Training Error :')
print('Mean Absolute Error : ',training_MAE)
print('Mean Square Error : ',training_MSE)
print('Root Mean Square Error : ',training_RMSE)

Training Error :
Mean Absolute Error :  533.2902743367749
Mean Square Error :  522517.1026142066
Root Mean Square Error :  722.8534447688595


In [ ]:
testing_MAE = sum([abs(test_pred[i] - Ytest[i]) for i in range(len(Ytest))]) / len(Ytest)
testing_MSE = sum([(test_pred[i] - Ytest[i]) ** 2 for i in range(len(Ytest))]) / len(Ytest)
testing_RMSE = testing_MSE ** 0.5
print('Testing Error :')
print('Mean Absolute Error : ',testing_MAE)
print('Mean Square Error : ',testing_MSE)
print('Root Mean Square Error : ',testing_RMSE)

Testing Error :
Mean Absolute Error :  499.61704252688105
Mean Square Error :  473815.55656171305
Root Mean Square Error :  688.3426156803841


<b><h3>Deployment</h3></b>

In [ ]:
x = float(input('Enter Load on Previous Day : '))
x = (x - Xmin) / (Xmax - Xmin)
prediction = m * x + c
prediction = (prediction * (Ymax - Ymin)) + Ymin
print('Load on Present Day :', prediction)

Enter Load on Previous Day : 9102
Load on Present Day : 8274.858567029349
